# Deploying an End-to-­End Machine Learning Solution on Kubeflow Pipelines

A Kubeflow pipeline component is an implementation of a pipeline task. A component is a step in the workflow. Each task takes one or more artifacts as input and may produce one or more artifacts as output.

Each component usually includes two parts:
- Client code: The code that talks to endpoints to submit jobs, for example, code to connect with the Google Cloud Machine Learning Engine.
- Runtime code: The code that does the actual job and usually runs in the cluster, for example, the code that prepares the model for training on Cloud MLE.

A component consists of an interface (inputs/outputs), the implementation (a Docker container image and command-line arguments), and metadata (name, description).

# Overview of a Simple End-to-End Solution Pipeline

In this simple example, we will implement a deep neural regressor network to predict the closing prices of Bitcoin crypto-currency. The machine learning code itself is pretty basic as it is not the focus of this article. The goal here is to orchestrate a machine learning engineering solution using microservice architectures on Kubernetes with Kubeflow Pipelines. The code for this chapter is in the book code repository. Clone the repository from the GCP Cloud Shell.

The pipeline consists of the following components:
1. Move raw data hosted on GitHub to a storage bucket.
2. Transform the dataset using Google Dataflow.
3. Carry out hyper-parameter training on Cloud Machine Learning Engine.
4. Train the model with the optimized hyper-parameters.
5. Deploy the model for serving on Cloud MLE.

# Create a Container Image for Each Component

First, we’ll package the client and runtime code into a Docker image. This image also contains the secure service account key to authenticate against GCP. For example, the component to transform the dataset using Dataflow has the following files built into its image:

- Dockerfile: Dockerfile to build the Docker image.
- build.sh: Script to initiate the container build and upload to Google Container Registry.
- dataflow_transform.py: Code to run the beam pipeline on Cloud Dataflow.
- service_account.json: Secure key to authenticate container on GCP.
- local_test.sh: Script to run the image pipeline component locally.

# Build Containers Before Uploading to Kubeflow Pipelines

Before uploading the pipeline to Kubeflow Pipelines, be sure to build the component containers so that the latest version of the code is packaged and uploaded as images to the container registry. The code provides a handy bash script to build all containers.

# Compile the Pipeline Using the Kubeflow Pipelines DSL Language

The pipeline code contains a specification on how the components interact with one another. Each component has an output that serves as an input to the next component in the pipeline. The Kubeflow pipeline DSL language dsl-compile from the Kubeflow Pipelines SDK is used to compile the pipeline code in Python for upload to Kubeflow Pipelines.

Ensure the Kubeflow Pipelines SDK is installed on the local machine by running

`# install kubeflow pipeline sdk
pip install https://storage.googleapis.com/ml-pipeline/release/0.1.12/kfp.
tar.gz --upgrade
# verify the install
which dsl-compile`

Compile the pipeline by running

`# compile the pipeline
python3 [path/to/python/file.py] [path/to/output/tar.gz]`

For the sample code, we used

`python3 crypto_pipeline.py crypto_pipeline.tar.gz`

# Upload and Execute the Pipeline to Kubeflow Pipelines

The following steps upload and execute the compiled pipeline on Kubeflow Pipelines:

1. Upload the pipeline to Kubeflow Pipelines (Figure 47-1).

> **Figure 47-1: _Upload the compiled pipeline to Kubeflow Pipelines_**<br><img width="300" src="data:image/png;base64, ..." />

2. Click the pipeline to see the static graph of the flow (Figure 47-2).

> **Figure 47-2: _Pipeline summary graph_**<br><img width="300" src="data:image/png;base64, ..." />

3. Create an Experiment and run to execute the pipeline (Figure 47-3).

> **Figure 47-3: _Create and run the Experiment_**<br><img width="300" src="data:image/png;base64, ..." />

4. Completed Pipeline run (Figure 47-4).

> **Figure 47-4: _Completed Pipeline run_**<br><img width="300" src="data:image/png;base64, ..." />

Completed Dataflow Pipeline: The completed run of the second component of the Pipeline, which is to transform the dataset with Cloud Dataflow, is illustrated in Figure 47-5.

> **Figure 47-5: _Completed Dataflow run_**<br><img width="300" src="data:image/png;base64, ..." />

Deployed model on Cloud MLE: The deployed model on Cloud MLE, which is the fifth component of the Pipeline, is illustrated in Figure 47-6.

> **Figure 47-6: _Deployed model on Cloud MLE_**<br><img width="300" src="data:image/png;base64, ..." />

**Note**: Always remember to clean up cloud resources when they are no longer  needed.

Delete Kubeflow: Run the script to delete the deployment.

`# navigate to kubeflow app
cd ${KFAPP}
# run script to delete the deployment
${KUBEFLOW_SRC}/scripts/kfctl.sh delete all`

Delete the Kubernetes cluster: Replace name with your own cluster name.

`# delete the kubernetes cluster
gcloud container clusters delete ekaba-gke-cluster`

This chapter covered building an end-to-end machine learning product as a containerized application on Kubernetes with Kubeflow and Kubeflow pipelines. Again, the code for this chapter may be accessed by cloning the book repository to the Cloud Shell.

This concludes this book.